In [12]:
import pandas as pd

# Wczytanie pliku CSV
file_path = "ex_ret.csv"  # Upewnij się, że plik jest w tym samym folderze co skrypt
df = pd.read_csv(file_path, sep=";", index_col=0)

# Konwersja indeksu na format daty dla poprawnego sortowania
df.index = pd.to_datetime(df.index, format="%d.%m.%Y", errors='coerce')

# Usunięcie pustych wierszy (gdzie wszystkie wartości to NaN)
df = df.dropna(how="all")

# Sprawdzenie, czy DataFrame nie jest pusty
if df.empty:
    print("❌ Błąd: Plik CSV jest pusty lub format danych jest niepoprawny.")
else:
    # Lista na wyniki dla każdego miesiąca
    results_list = []

    for selected_month in df.index.unique():  # Iteracja po unikalnych miesiącach
        # Pobranie zwrotów dla wybranego miesiąca
        returns_selected_month = df.loc[selected_month]

        # Sortowanie zwrotów, aby znaleźć najlepsze i najgorsze
        sorted_returns = returns_selected_month.sort_values(ascending=False)

        # Pobranie tylko **20% najlepszych i 20% najgorszych** akcji
        num_stocks = len(sorted_returns)
        top_20_percent = sorted_returns.head(int(num_stocks * 0.2)).index.tolist()
        bottom_20_percent = sorted_returns.tail(int(num_stocks * 0.2)).index.tolist()

        # Zapisanie wyników do listy
        results_list.append(pd.DataFrame({
            "Date": [selected_month.strftime('%Y-%m')],
            "Top Performers": [", ".join(top_20_percent)],
            "Bottom Performers": [", ".join(bottom_20_percent)]
        }))

    # Połączenie wyników w jeden DataFrame
    results_df = pd.concat(results_list, ignore_index=True).set_index("Date")

    # Wyświetlenie tabeli
    print(results_df)

    # Zapisanie wyników do pliku CSV
    output_file = "monthly_top_bottom_20_percent_1_month.csv"
    results_df.to_csv(output_file, sep=",", encoding="utf-8")

    print(f"\n✅ Wyniki zapisano do pliku: {output_file}")


                                            Top Performers  \
Date                                                         
2002-02   EMN, SJM, AVY, DHI, GL, IEX, SNA, ATO, PPG, HBAN   
2002-03  ESS, DTE, STT, CINF, INCY, TTWO, PPG, BAX, ATO...   
2002-04  EIX, CINF, MHK, ROK, FRT, BRO, AVY, HOLX, ED, ...   
2002-05  ADM, PPG, AFL, BRO, IEX, BXP, EMN, AEE, APD, JBHT   
2002-06  PPG, JBHT, DHI, ESS, AOS, SJM, MAA, ATO, CINF,...   
...                                                    ...   
2023-08  BRO, AFL, AVY, DOV, ROP, INCY, CHD, IEX, HBAN, FE   
2023-09  AFL, JBHT, DHI, NUE, AOS, NTRS, IEX, EIX, ATO,...   
2023-10      GL, AOS, FE, ED, SWK, CMS, RHI, AEE, ESS, AFL   
2023-11  ROP, POOL, MHK, RHI, VMC, DOV, AOS, ROL, COO, DTE   
2023-12  AOS, DOV, SWK, JBHT, MAA, BAX, RHI, FRT, CMI, IEX   

                                         Bottom Performers  
Date                                                        
2002-02    JBHT, FDS, AFL, FE, RHI, ADM, BAX, ED, ROL, LNC  
2002-03   

In [13]:
import pandas as pd

# Load the precomputed list of Top and Bottom performers
file_path_top_bottom = "monthly_top_bottom_20_percent_1_month.csv"
df_top_bottom = pd.read_csv(file_path_top_bottom, sep=",", index_col=0)

# Convert index to datetime to match the format in ret.csv
df_top_bottom.index = pd.to_datetime(df_top_bottom.index, format="%Y-%m", errors='coerce')

# Load the returns dataset (ret.csv)
file_path_returns = "ex_ret.csv"
df_returns = pd.read_csv(file_path_returns, sep=";", index_col=0)

# Convert index to datetime (DD.MM.YYYY format)
df_returns.index = pd.to_datetime(df_returns.index, format="%d.%m.%Y", errors='coerce')

# Convert returns index to monthly format (YYYY-MM) to match df_top_bottom
df_returns.index = df_returns.index.to_period("M")

# 🟢 🔹 **Usunięcie % i konwersja na float** 🔹 🟢
df_returns = df_returns.replace("%", "", regex=True).astype(str).applymap(lambda x: x.replace(",", "."))
df_returns = df_returns.apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN values
df_returns = df_returns.dropna(how="all")

# Initialize list for results
avg_results_list = []

# Iterate through available months
for selected_month in df_top_bottom.index:
    period_month = selected_month.to_period("M")  # Convert to Period format

    if period_month in df_returns.index:
        # Retrieve the list of top and bottom performers
        top_firms = df_top_bottom.loc[selected_month, "Top Performers"]
        bottom_firms = df_top_bottom.loc[selected_month, "Bottom Performers"]

        # Ensure lists are properly formatted
        if isinstance(top_firms, str) and isinstance(bottom_firms, str):
            top_firms = [firm.strip() for firm in top_firms.split(",")]
            bottom_firms = [firm.strip() for firm in bottom_firms.split(",")]
        else:
            continue  # Skip iteration if data is not properly formatted

        # Get the returns for these firms from the returns dataset
        selected_returns_month = df_returns.loc[period_month]

        # Extract returns of top and bottom firms (only if they exist in returns dataset)
        valid_top_firms = [firm for firm in top_firms if firm in selected_returns_month]
        valid_bottom_firms = [firm for firm in bottom_firms if firm in selected_returns_month]

        if valid_top_firms and valid_bottom_firms:
            # 🟢 🔹 **Usunięcie błędnych wartości i konwersja na float** 🔹 🟢
            top_returns = selected_returns_month[valid_top_firms].apply(pd.to_numeric, errors='coerce').mean()
            bottom_returns = selected_returns_month[valid_bottom_firms].apply(pd.to_numeric, errors='coerce').mean()

            # Store the result
            avg_results_list.append(pd.DataFrame({
                "Date": [selected_month.strftime('%Y-%m')],
                "Avg Top Performers": [top_returns],
                "Avg Bottom Performers": [bottom_returns]
            }))

# Combine results into a single DataFrame
if avg_results_list:
    avg_results_df = pd.concat(avg_results_list, ignore_index=True).set_index("Date")
    # Save results to CSV
    output_file = "avg_monthly_top_bottom_20_percent_1_month.csv"
    avg_results_df.to_csv(output_file, sep=",", encoding="utf-8")

    print(f"\n✅ Results saved to: {output_file}")
    print(avg_results_df)
else:
    print("\n❌ No matching data found. Check the formatting of the files.")


C:\Users\adamr\AppData\Local\Temp\ipykernel_27792\2713951397.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_returns = df_returns.replace("%", "", regex=True).astype(str).applymap(lambda x: x.replace(",", "."))



✅ Results saved to: avg_monthly_top_bottom_20_percent_1_month.csv
         Avg Top Performers  Avg Bottom Performers
Date                                              
2002-02               6.572                 -3.413
2002-03               8.177                 -4.014
2002-04               5.994                 -8.817
2002-05               6.174                 -6.739
2002-06               4.414                 -6.740
...                     ...                    ...
2023-08              -0.576                 -0.892
2023-09              -6.866                 -7.044
2023-10               2.733                 -6.483
2023-11               8.569                  4.788
2023-12               7.845                 -2.303

[263 rows x 2 columns]


In [14]:
import pandas as pd

# Load the precomputed list of Top and Bottom performers
file_path_top_bottom = "monthly_top_bottom_20_percent_1_month.csv"
df_top_bottom = pd.read_csv(file_path_top_bottom, sep=",", index_col=0)

# Convert index to datetime to match the format in ret.csv
df_top_bottom.index = pd.to_datetime(df_top_bottom.index, format="%Y-%m", errors='coerce')

# Load the returns dataset (ret.csv)
file_path_returns = "ret.csv"
df_returns = pd.read_csv(file_path_returns, sep=";", index_col=0)

# Convert index to datetime (DD.MM.YYYY format)
df_returns.index = pd.to_datetime(df_returns.index, format="%d.%m.%Y", errors='coerce')

# Convert returns index to monthly format (YYYY-MM) to match df_top_bottom
df_returns.index = df_returns.index.to_period("M")

# 🟢 🔹 **Usunięcie % i konwersja na float** 🔹 🟢
df_returns = df_returns.replace("%", "", regex=True).astype(str).applymap(lambda x: x.replace(",", "."))
df_returns = df_returns.apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN values
df_returns = df_returns.dropna(how="all")

# Initialize list for results
avg_results_list = []

# Iterate through available months
for selected_month in df_top_bottom.index:
    period_month = selected_month.to_period("M")  # Convert to Period format

    if period_month in df_returns.index:
        # Retrieve the list of top and bottom performers
        top_firms = df_top_bottom.loc[selected_month, "Top Performers"]
        bottom_firms = df_top_bottom.loc[selected_month, "Bottom Performers"]

        # Ensure lists are properly formatted
        if isinstance(top_firms, str) and isinstance(bottom_firms, str):
            top_firms = [firm.strip() for firm in top_firms.split(",")]
            bottom_firms = [firm.strip() for firm in bottom_firms.split(",")]
        else:
            continue  # Skip iteration if data is not properly formatted

        # Get the returns for these firms from the returns dataset
        selected_returns_month = df_returns.loc[period_month]

        # Extract returns of top and bottom firms (only if they exist in returns dataset)
        valid_top_firms = [firm for firm in top_firms if firm in selected_returns_month]
        valid_bottom_firms = [firm for firm in bottom_firms if firm in selected_returns_month]

        if valid_top_firms and valid_bottom_firms:
            # 🟢 🔹 **Usunięcie błędnych wartości i konwersja na float** 🔹 🟢
            top_returns = selected_returns_month[valid_top_firms].apply(pd.to_numeric, errors='coerce').mean()
            bottom_returns = selected_returns_month[valid_bottom_firms].apply(pd.to_numeric, errors='coerce').mean()

            # Store the result
            avg_results_list.append(pd.DataFrame({
                "Date": [selected_month.strftime('%Y-%m')],
                "Avg Top Performers": [top_returns],
                "Avg Bottom Performers": [bottom_returns]
            }))

# Combine results into a single DataFrame
if avg_results_list:
    avg_results_df = pd.concat(avg_results_list, ignore_index=True).set_index("Date")
    # Save results to CSV
    output_file = "avg_monthly_top_bottom_20_percent_1_month_ret.csv"
    avg_results_df.to_csv(output_file, sep=",", encoding="utf-8")

    print(f"\n✅ Results saved to: {output_file}")
    print(avg_results_df)
else:
    print("\n❌ No matching data found. Check the formatting of the files.")


C:\Users\adamr\AppData\Local\Temp\ipykernel_27792\2246357801.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_returns = df_returns.replace("%", "", regex=True).astype(str).applymap(lambda x: x.replace(",", "."))



✅ Results saved to: avg_monthly_top_bottom_20_percent_1_month_ret.csv
         Avg Top Performers  Avg Bottom Performers
Date                                              
2002-02               6.702                 -3.283
2002-03               8.307                 -3.884
2002-04               6.144                 -8.667
2002-05               6.314                 -6.599
2002-06               4.544                 -6.610
...                     ...                    ...
2023-08              -0.126                 -0.442
2023-09              -6.436                 -6.614
2023-10               3.203                 -6.013
2023-11               9.009                  5.228
2023-12               8.275                 -1.873

[263 rows x 2 columns]
